In [1]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def Forward(A0, W1, b1, W2, b2): # for 3 layers
    z1 = np.dot(A0, W1) + b1
    A1 = z1*(z1 > 0)
    z2 = np.dot(A1, W2) + b2
    A2 = z2 * (z2 > 0)
    return z1, A1, z2, A2

In [3]:
def Backpropagation(A0, W1, Z1, A1, W2, Z2, A2, T):  # for 3 layers
    # For second layer
    Delta2 = T - A2
    Err2 = Delta2 * (Z2>0)
    dw2 = np.dot(A1.T, Err2)
    db2 = Err2.sum(axis=0)
    
    # For First layer
    Delta1 = np.dot(Err2, W2.T)
    Err1 = Delta1 * (Z1>0)
    dw1 = np.dot(A0.T, Err1)
    db1 = Err1.sum(axis=0)
    return dw1, db1, dw2, db2

In [4]:
def NN_create_Wb(A0, HL):
    W = []
    b = []
    for i in range(len(HL)):
        if i == 0:
            W_i = np.random.randn(A0.shape[1], HL[0])/np.sqrt(HL[0])
        else:
            W_i = np.random.randn(HL[i-1], HL[i])/np.sqrt(HL[i])
        b_i = np.random.randn(1, HL[i])/np.sqrt(HL[i])
        W.append(W_i)
        b.append(b_i)
    return W, b

In [5]:
def NN_compute_Err_i(Delta_i, Z_i, A_i, af):
    if type(af) == str:
        if af == 'Sigmoid':
            Error_i = Delta_i * A_i * (1 - A_i)
        elif af == 'ReLU':
            Error_i = Delta_i * (Z_i > 0)
        elif af == 'Gaussian':
            Error_i = Delta_i * (-2*Z_i) * np.exp(-(Z_i**2))
    elif type(af) == list:
        if af[0] == 'PReLU':
            Error_i = Delta_i*(Z_i > 0) + Delta_i*af[1]*(Z_i <= 0)
    return Error_i

In [6]:
def NN_backpropagation(TypeOfPrediction, A0, W, b, Z, A, T, AF):
    SW = []
    Sb = []
    for i in range(len(AF) - 1, -1, -1):
        if i == len(AF) - 1:
            if TypeOfPrediction == 'Regression':
                Delta_i = T - A[i]
            elif TypeOfPrediction == 'Classification':
                Delta_i = T/A[i] - (1 - T)/(1 - A[i])
        else:
            Delta_i = np.dot(Err_i, W[i + 1].T)
        Err_i = NN_compute_Err_i(Delta_i, Z[i], A[i], AF[i])
        if i != 0:
            SW_i = np.dot(A[i - 1].T, Err_i)
        else:
            SW_i = np.dot(A0.T, Err_i)
        Sb_i = Err_i.sum(axis = 0)
        SW.append(SW_i)
        Sb.append(Sb_i)
    return SW[::-1], Sb[::-1]

In [10]:
def NN_find_Wb(TypeOfPrediction, A0, T, HL, AF, epoch = 1000, lr = 0.01, target_error = 0, plot = True, width = 9, height = 4.5):
    N = A0.shape[0]
    W, b = NN_create_Wb(A0, HL)
    error_list = []
    if epoch == 'inf':
        while True:
            Z, A = NN_find_ZA(A0, W, b, AF)
            error = find_sse(T, A[-1])
            error_list.append(error)
            if error <= target_error:
                break
            SW, Sb = NN_backpropagation(TypeOfPrediction, A0, W, b, Z, A, T, AF)
            for k in range(len(HL)):
                W[k] = W[k] + (lr/N)*SW[k]
                b[k] = b[k] + (lr/N)*Sb[k]
    elif epoch != 'inf':
        for i in range(epoch):
            Z, A = NN_find_ZA(A0, W, b, AF)
            error = find_see(T, A[-1])
            error_list.append(error)
            if error <= target_error:
                break
            SW, Sb = NN_backpropagation(TypeOfPrediction, A0, W, b, Z, A, T, AF)
            for k in range(len(HL)):
                W[k] = W[k] + (lr/N)*SW[k]
                b[k] = b[k] + (lr/N)*Sb[k]
    if plot == True:
        fig = plt.figure(figsize = (width, height))
        plt.plot(error_list, '#1f77b4', label = 'Train Error')
        plt.legend()
    return W, b, error_list

In [12]:
def NN_find_Wb_optimize(TypeOfPrediction, A0Train, TTrain, A0Validation, TValidation, HL, AF, \
                        epoch = 1000, lr = 0.01, target_error = 0, plot = True, width = 9, height = 4.5):
    N = A0.shape[0]
    W, b = NN_create_Wb(A0, HL)
    best_W = W
    best_b = b
    train_error_list = []
    validation_error_list = []
    min_validation_error = np.inf
    for i in range(epoch):
        ZTrain, ATrain = NN_find_ZA(A0Train, W, b, AF)
        ZValidation, AValidation = NN_find_ZA(A0Validation, W, b, AF)
        train_error = find_sse(TTrain, ATrain[-1])
        validation_error = find_sse(TValidation, AValidation[-1])
        train_error_list.append(train_error)
        validation_error_list.append(validation_error)
        if validation_error <= target_error:
            best_W = W
            best_b = b
            break
        if validation_error < min_validation_error:
            min_validation_error = validation_error
            best_W = W
            best_b = b
        SW, Sb = NN_backpropagation(TypeOfPrediction, A0Train, W, b, ZTrain, ATrain, TTrain, AF)
        for k in range(len(HL)):
            W[k] = W[k] + (lr/N)*SW[k]
            b[k] = b[k] + (lr/N)*Sb[k]
    if plot == True:
        fig = plt.figure(figsize = (width, height))
        plt.plot(train_error_list, '1f77b4', label = 'Train Error')
        plt.plot(validation_error_list, 'ff7f04', label = 'Validation Error')
        plt.legend()
    return best_W, best_b, train_error_list, validation_error_list

In [14]:
def NN_compute_A_i(Z_i, af):
    if type(df) == str:
        if af == 'Sigmoid':
            A_i = 1/(1 + np.exp(-Z_i))
        elif af == 'ReLU':
            A_i = Z_i * (Z_i > 0)
        elif af == 'Gaussian':
            A_i = np.exp(-(Z_i**2))
    elif type(af) == list:
        if af[0] == 'PReLU':
            A_i = Z_i*(Z_i > 0) + Z_i*af[1]*(Z_i <= 0)
    return A_i

In [15]:
def NN_find_ZA(A0, T, W, b, AF, plot = True, sep = False, width = 9, height = 4.5, dataset = ''):
    Z = []
    A = []
    for i in range(len(AF)):
        if i == 0:
            Z_i = np.dot(A0, W[i]) + b[i]
        else:
            Z_i = np.dot(A[-1], W[i]) + b[i]
        A_i = NN_compute_A_i(Z_i, AF[i])
        Z.append(Z_i)
        A.append(A_i)
    if sep == False:
        fig = plt.figure(figsize = (width, height))
        plt.plot(T, '#1f77b4', label = 'True ' + dataset + ' Value')
        plt.plot(A[-1], '#ff7f04', label = 'Predicted ' + dataset + ' Value')
        plt.legend()
    elif sep == True:
        fig = plt.figure(figsize = (width*2, height))
        plt.subplot(1, 2, 1)
        plt.plot(T, '#1f77b4', label = 'True ' + dataset + ' Value')
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.plot(A[-1], '#ff7f04', label = 'Predicted ' + dataset + ' Value')
        plt.legend()
    return Z, A

In [16]:
def NN_find_Wb_ZA(TypeOfPrediction, A0, T, HL, AF, epoch = 1000, lr = 0.01, target_error = 0, \
                  plot = True, error_plot = True, sep = False, width = 9, height = 4.5, dataset = ''):
    
    W, b, error_list = NN_find_Wb(TypeOfPrediction, A0, T, HL, AF, epoch = epoch, \
                                  lr = lr, target_error = target_error, plot = error_plot, width = width, height = height)
    
    Z, A = NN_find_ZA(A0, T, W, b, AF, plot = plot, sep = sep, width = width, height = height, dataset = dataset)
    return W, b, error_list, Z, A

In [17]:
def NN_find_Wb_optimize_ZA(TypeOfPrediction, A0Train, TTrain, A0Validation, TValidation, HL, AF, \
                           epoch = 1000, lr = 0.01, target_error = 0, plot = True, error_plot = True, \
                           sep = False, width = 9, height = 4.5, dataset = ''):
    
    W, b, train_error_list, validation_error_list = NN_find_Wb_optimize(TypeOfPrediction, A0Train, TTrain, \
                                                                        A0Validation, TValidation, HL, AF, \
                                                                        epoch = epoch, lr = lr, \
                                                                        target_error = target_error, \
                                                                        plot = error_plot, width = width, height = height)
    
    ZTrain, ATrain = NN_find_ZA(A0Train, TTrain, W, b, AF, plot = plot, sep = sep, width = width, \
                                height = height, dataset = 'Train')
    
    ZValidation, AValidation = NN_find_ZA(A0Validation, TValidation, W, b, AF, plot = plot, \
                                          sep = sep, width = width, height = height, dataset = 'Validation')
    return W, b, train_error_list, validation_error_list, ZTrain, ATrain, ZValidation, AValidation

In [18]:
def NN_find_Wb_ZATrain_ZATest(TypeOfPrediction, A0Train, TTrain, A0Test, TTest, HL, AF, \
                              epoch = 1000, lr = 0.01, target_error = 0, plot = True, \
                              error_plot = True, sep = False, width = 9, height = 4.5, dataset = ''):
    
    W, b, train_error_list, validation_error_list = NN_find_Wb(TypeOfPrediction, A0Train, TTrain, \
                                                               HL, AF, epoch = epoch, lr = lr, target_error = target_error, \
                                                               plot = error_plot, width = width, height = height)
    
    ZTrain, ATrain = NN_find_ZA(A0Train, TTrain, W, b, AF, plot = plot, sep = sep, width = width, \
                                height = height, dataset = 'Train')
    
    ZTest, ATest = NN_find_ZA(A0Test, TTest, W, b, AF, plot = plot, sep = sep, width = width, height = height, dataset = 'Test')
    return W, b, train_error_list, validation_error_list, ZTrain, ATrain, ZTest, ATest

In [20]:
def NN_find_Wb_optimize_ZATrain_ZAValidation_ZATest(TypeOfPrediction, A0Train, TTrain, A0Validation, \
                                                    TValidation, A0Test, TTest, HL, AF, epoch = 1000, lr = 0.01, \
                                                    target_error = 0, plot = True, error_plot = True, \
                                                    sep = False, width = 9, height = 4.5, dataset = ''):
    
    W, b, train_error_list, validation_error_list = NN_find_Wb_optimize(TypeOfPrediction, A0Train,
                                                                        TTrain, A0Validation, TValidation,
                                                                        HL, AF, epoch = epoch, lr = lr,
                                                                        target_error = target_error,
                                                                        plot = error_plot, width = width, height = height)
    
    ZTrain, ATrain = NN_find_ZA(A0Train, TTrain, W, b, AF, plot = plot, sep = sep, width = width, \
                                height = height, dataset = 'Train')
    ZValidation, AValidation = NN_find_ZA(A0Validation, TValidation, W, b, AF, plot = plot, sep = sep, \
                                          width = width, height = height, dataset = 'Validation')
    ZTest, ATest = NN_find_ZA(A0Test, TTest, W, b, AF, plot = plot, sep = sep, width = width, height = height, dataset = 'Test')
    return W, b, train_error_list, validation_error_list, ZTrain, ATrain, ZValidation, AValidation, ZTest, ATest